# Chapter 12. Inheritance: For Good or For Worse

# Subclassing Built-In Types is Tricky
- CPython doesn't call special methods overridden by user-defined classes
    - Subclassing built-in types (e.g, `dict`, `list`, `str`) directly is error-prone because the built-in methods mostly ignore user-defined overrides. -> should use `UserDict`, `UserList`, `UserString`, `MutableMapping` in `collections` module

In [9]:
# built-in dict ignores overriden __setitem__
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
# built-in dict ignores overriden __getitem__
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [6]:
ad = AnswerDict(a='foo')
ad

{'a': 42}

In [7]:
d = {}
d.update(ad)
d['a']

'foo'

In [8]:
d

{'a': 'foo'}

In [10]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [11]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [12]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [13]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [14]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [15]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [16]:
d = {}
d.update(ad)
d['a']

42

In [17]:
d

{'a': 42}

# Multiple Inheritance and Method Resolution Order
- conflicts of attribute names
- diamond problem
    - Method Resolution Order; MRO follows the order of inheritance
        - e.g; When D(B, C), the method of C is called.
        - `__mro__` can be used to check MRO

In [21]:
# A diamond problem example
class A:
    def ping(self):
        print('ping:', self)
    
class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()  # A.ping(self)
        self.pong()
        super().pong()
        C.pong(self)

In [19]:
d = D()
d.pong()

pong <__main__.D object at 0x10684fe10>


In [22]:
C.pong(d)

PONG: <__main__.D object at 0x10684fe10>


In [23]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [24]:
A.ping()

TypeError: ping() missing 1 required positional argument: 'self'

In [25]:
A.ping(A)

ping: <class '__main__.A'>


In [26]:
A.ping('a')

ping: a


In [27]:
d = D()
d.ping()

ping: <__main__.D object at 0x106c04a90>
post-ping: <__main__.D object at 0x106c04a90>


In [28]:
d.pingpong()

ping: <__main__.D object at 0x106c04a90>
post-ping: <__main__.D object at 0x106c04a90>
ping: <__main__.D object at 0x106c04a90>
pong: <__main__.D object at 0x106c04a90>
pong: <__main__.D object at 0x106c04a90>
PONG: <__main__.D object at 0x106c04a90>


# Multiple Inheritance in the Real World

## Tips for multiple inheritance
1. Distinguish Interface Inheritance from Implementation Inheritance
    - Inheritance of interface creates a subtype, implying an "is-a relationship.
    - Inheritance of implementation avoids code duplication by reuse.
1. Make Insterfaces Explicih with ABCs
1. Use `Mixin`s for Code Reuse
    - Conceptually, a mixin class doesn't define a new type; it merely bundles methods for reuse.
    - A mixin should never be instantiated, and concrete classes should not inherit only from a mixin.
1. Make `Mixin`s Explicit by Naming
    - There is no fomal representation for mixin, so it's highly recommended to name those with `...Mixin` suffix.
1. An ABC May Also Be A Mixin; The Reverse is Not True
    - The concrete methods implemented in an ABC should only collaborate with methods of the same ABC and its superclasses.
1. Don't Subclass from More Than One Concrete Class
    - i.e, all but one of the superclasses of a concrete class should be ABCs or mixins.
1. Provide Aggregate Classes to Users
    - e.g, empty but useful as interface for users
```python
class Widget(BaseWidget, Pack, Place, Grid):  
    pass
```  

1. "Favor Object Composition Over Class Inheritance."